In [1]:
import torch.nn as nn
import torch
import gc

# Paquete de optimizadores
import torch.optim as optim

from sklearn.metrics import accuracy_score

In [2]:
# Realiza el entrenamiento de una epoca
# Argumentos:
#       - model: modelo que queremos entrenar
#       - train_dataloader: lotes de entrenamiento
#       - optm: optimizador del modelo empleado
#       - loss_criterion: define la funcion de perdida
#       - device: cpu o cuda, define si el entrenamiento se hace en la cpu o gpu


def train_unet(model, train_dataloader, optm, loss_criterion, device):
    
    # Lista para las perdidas de cada época
    train_loss = []
    
    # Para calcular la precisión
    correct = 0
    total = 0
    
    # Modo de entrenamiento del modelo
    model.train()
    
    # Iteramos sobre los lotes de train
    for images, masks in train_dataloader:
        
        # Permutamos las dimensiones de las imágenes a (N, C, H, W)
        images = images.permute(0, 3, 1, 2)
        
        # Movemos al device las imagenes y las Ground Truth
        images = images.to(device, dtype=torch.float)
        masks = masks.to(device, dtype=torch.long)
        
        # El modelo realiza la inferencia para las imagenes del lote
        outputs = model(images)
        
        # Calculamos la pérdida
        loss = loss_criterion(outputs, masks)
        
        # Limpiamos los gradientes del optimizador
        optm.zero_grad()
        
        # Retropropagación y actualizamos los pesos del modelo
        loss.backward()
        optm.step()
  
        # predicted --> tensor que contiene la clase predicha con la mayor probabilidad para cada píxel en la salida del modelo
        _, predicted = torch.max(outputs.data, 1)
        
        total += masks.nelement()  # Número total de píxeles en la máscara
        
        # eq: compara elemento a elemento la prediccion con los valores reales de la imagen Ground Truth. generando un tensor de booleanos
        # sum: suma los elementos true del tensor, es decir, los aciertos
        correct += predicted.eq(masks.data).sum().item() 
        
        # Liberamos memoria
        del images, masks
        gc.collect()
        
        # Añadimos la prdida a la lista
        train_loss.append(loss.item())

    # Precisión del modelo
    accuracy = round((correct / total)*100,2)  
    
    return train_loss, accuracy